# Setting up the *p-Arm* configuration

Through this guide the basic params for the *p-Arm* manipulator will be set-up.

The required options are:

+ The **end-effector** length.
+ The **lower arm** length.
+ The **upper arm** lenght.
+ The **base** height.
+ The **little deviation** from base.

All these information conforms de *Denavit-Hartenberg* table which allows us to understand how the movement of each motor (joint) will affect the final position. For this arm in particular, it is working on the $XZ$ axis, so the first $a_i$ is swapped with the correspondant $d_i$.

In addition, we have to know the angle in between each joint. Having a look to the picture, we know that only the first segment and the second one have an angle different to '0'.

![uArm sizes and configuration](https://github.com/UPM-Robotics/uarm/raw/master/docs/memory/images/sizes.png "uArm sizes")

Finally, as the *end-effector* position is always parallel to the floor (this means its angle is always $π$), we can remove it from the table and include it as a traslation in the '$Z$' axis.

So then, the final table results as:

| $i$ | $θ_i$ | $d_i (mm.)$ | $a_i (mm.)$ | $α_i$ |
|---|:--:|----------|----------|----|
| $1$ | $θ_1$ |       $a_1$ |       $d_1$ | $α_1$ |
| $2$ | $θ_2$ |        $0$ |       $a_2$ | $α_2$ |
| $3$ | $θ_3$ |        $0$ |       $a_3$ |  $0$ |

## Install the required packages

For running, it is necessary to install the `sympy` module (see [sympy on GitHub](https://github.com/sympy/sympy)) and `numpy` ([numpy](https://numpy.org/)).

In [1]:
pip install sympy numpy

Note: you may need to restart the kernel to use updated packages.


Now, import `DHTable` from the `manipulator` package and start defining things.

As we saw in the previous table, we need three symbols: $θ_1$, $θ_2$, $θ_3$.

In [8]:
from manipulator import (
    DHTable,
    pi,
    Manipulator
)
from sympy import symbols

parm_table = DHTable()
t1, t2, t3 = symbols("theta_1 theta_2 theta_3")

Now, as we have our three symbols, we must fill the other values necessary to the table to work properly. As shown in the table, we need to define:

+ $a_1$, $a_2$, $a_3$.
+ $d_1$.
+ $α_1$, $α_2$.
+ $T_X$, $T_Z$.

The latest ones ($T_x$, $T_z$) stand for the traslation in the $X$ axis and traslation in $Z$ axis.

In [9]:
# Different arm lengths
## For specific lengths
# a1 = 106.6
# a2 = 142.07
# a3 = 158.81
## For symbolic calculus
a1, a2, a3 = symbols("a_1 a_2 a_3")

# The deviation present in between joints
## For specific lengths
# d1 = 13.2
## For symbolic calculus
d1 = symbols("d_1")

# The traslations in both X and Z axis
## For specific lengths
# Tx = 44.5
# Tz = -13.2
## For symbolic calculus
Tx, Tz = symbols("T_X T_Z")

# The angle in between joints
alpha_1 = (pi / 2)
alpha_2 = pi

Then, we can define the `DHTable` by using the `add` method, as follows:

In [11]:
## False for symbolic calculus
is_digits = False
parm_table.add(theta=t1, d=a1, a=d1, alpha=alpha_1, check_attrs=is_digits)\
          .add(theta=t2, d=0, a=a2, alpha=alpha_2, check_attrs=is_digits)\
          .add(theta=t3, d=0, a=a3, alpha=0, check_attrs=is_digits)
parm_table.Tx = Tx
parm_table.Tz = Tz

The generated table is:

In [12]:
print(parm_table)

   i         θᵢ     dᵢ     aᵢ      αᵢ
   1    theta_1    a_1    d_1    pi/2
   2    theta_2      0    a_2      pi
   3    theta_3      0    a_3       0



Now, we can declare a `Manipulator` with the obtained data, so we can directly obtain the forward kinematics matrices:

In [13]:
from pprint import pprint

pArm = Manipulator(params=parm_table)

# print each matrix data
for matrix in {'A01', 'A12', 'A23', 'A02', 'A03'}:
    print(matrix, end=':\n----------------------\n')
    pprint(pArm.direct_kinematics[matrix])

A02:
----------------------
Matrix([
[cos(theta_1)*cos(theta_2), sin(theta_2)*cos(theta_1), -sin(theta_1), (a_2*cos(theta_2) + d_1)*cos(theta_1)],
[sin(theta_1)*cos(theta_2), sin(theta_1)*sin(theta_2),  cos(theta_1), (a_2*cos(theta_2) + d_1)*sin(theta_1)],
[             sin(theta_2),             -cos(theta_2),             0,                a_1 + a_2*sin(theta_2)],
[                        0,                         0,             0,                                     1]])
A12:
----------------------
Matrix([
[cos(theta_2),  sin(theta_2),  0, a_2*cos(theta_2)],
[sin(theta_2), -cos(theta_2),  0, a_2*sin(theta_2)],
[           0,             0, -1,                0],
[           0,             0,  0,                1]])
A03:
----------------------
Matrix([
[cos(theta_1)*cos(theta_2 - theta_3), sin(theta_2 - theta_3)*cos(theta_1), -sin(theta_1), T_X + (a_2*cos(theta_2) + a_3*cos(theta_2 - theta_3) + d_1)*cos(theta_1)],
[sin(theta_1)*cos(theta_2 - theta_3), sin(theta_1)*sin(theta_2 - theta

In this case, we are interested in the matrix which is the multiplication of all the other matrices, in this case `A03`

In [14]:
pprint(pArm.direct_kinematics['A03'])

Matrix([
[cos(theta_1)*cos(theta_2 - theta_3), sin(theta_2 - theta_3)*cos(theta_1), -sin(theta_1), T_X + (a_2*cos(theta_2) + a_3*cos(theta_2 - theta_3) + d_1)*cos(theta_1)],
[sin(theta_1)*cos(theta_2 - theta_3), sin(theta_1)*sin(theta_2 - theta_3),  cos(theta_1),       (a_2*cos(theta_2) + a_3*cos(theta_2 - theta_3) + d_1)*sin(theta_1)],
[             sin(theta_2 - theta_3),             -cos(theta_2 - theta_3),             0,                T_Z + a_1 + a_2*sin(theta_2) + a_3*sin(theta_2 - theta_3)],
[                                  0,                                   0,             0,                                                                        1]])


In [15]:
print(pArm.to_latrix('p', 'A03'))

%\usepackage{amsmath}
\begin{pmatrix}
 \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2} - \theta_{3} \right)} &  \sin{\left(\theta_{2} - \theta_{3} \right)} \cos{\left(\theta_{1} \right)} &  - \sin{\left(\theta_{1} \right)} &  T_{X} + \left(a_{2} \cos{\left(\theta_{2} \right)} + a_{3} \cos{\left(\theta_{2} - \theta_{3} \right)} + d_{1}\right) \cos{\left(\theta_{1} \right)} \\
 \sin{\left(\theta_{1} \right)} \cos{\left(\theta_{2} - \theta_{3} \right)} &  \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} - \theta_{3} \right)} &  \cos{\left(\theta_{1} \right)} &  \left(a_{2} \cos{\left(\theta_{2} \right)} + a_{3} \cos{\left(\theta_{2} - \theta_{3} \right)} + d_{1}\right) \sin{\left(\theta_{1} \right)} \\
 \sin{\left(\theta_{2} - \theta_{3} \right)} &  - \cos{\left(\theta_{2} - \theta_{3} \right)} &  0 &  T_{Z} + a_{1} + a_{2} \sin{\left(\theta_{2} \right)} + a_{3} \sin{\left(\theta_{2} - \theta_{3} \right)} \\
 0 &  0 &  0 &  1 \\
\end{pmatrix}


More beautifully:

*Numeric one*:
\begin{pmatrix}
 \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2} - \theta_{3} \right)} &  \sin{\left(\theta_{2} - \theta_{3} \right)} \cos{\left(\theta_{1} \right)} &  - \sin{\left(\theta_{1} \right)} &  \left(142.07 \cos{\left(\theta_{2} \right)} + 158.81 \cos{\left(\theta_{2} - \theta_{3} \right)} + 13.2\right) \cos{\left(\theta_{1} \right)} + 44.5 \\
 \sin{\left(\theta_{1} \right)} \cos{\left(\theta_{2} - \theta_{3} \right)} &  \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} - \theta_{3} \right)} &  \cos{\left(\theta_{1} \right)} &  \left(142.07 \cos{\left(\theta_{2} \right)} + 158.81 \cos{\left(\theta_{2} - \theta_{3} \right)} + 13.2\right) \sin{\left(\theta_{1} \right)} \\
 \sin{\left(\theta_{2} - \theta_{3} \right)} &  - \cos{\left(\theta_{2} - \theta_{3} \right)} &  0 &  142.07 \sin{\left(\theta_{2} \right)} + 158.81 \sin{\left(\theta_{2} - \theta_{3} \right)} + 93.4 \\
 0 &  0 &  0 &  1 \\
\end{pmatrix}

*Parametrized one*:
\begin{pmatrix}
 \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2} - \theta_{3} \right)} &  \sin{\left(\theta_{2} - \theta_{3} \right)} \cos{\left(\theta_{1} \right)} &  - \sin{\left(\theta_{1} \right)} &  T_{X} + \left(a_{2} \cos{\left(\theta_{2} \right)} + a_{3} \cos{\left(\theta_{2} - \theta_{3} \right)} + d_{1}\right) \cos{\left(\theta_{1} \right)} \\
 \sin{\left(\theta_{1} \right)} \cos{\left(\theta_{2} - \theta_{3} \right)} &  \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} - \theta_{3} \right)} &  \cos{\left(\theta_{1} \right)} &  \left(a_{2} \cos{\left(\theta_{2} \right)} + a_{3} \cos{\left(\theta_{2} - \theta_{3} \right)} + d_{1}\right) \sin{\left(\theta_{1} \right)} \\
 \sin{\left(\theta_{2} - \theta_{3} \right)} &  - \cos{\left(\theta_{2} - \theta_{3} \right)} &  0 &  T_{Z} + a_{1} + a_{2} \sin{\left(\theta_{2} \right)} + a_{3} \sin{\left(\theta_{2} - \theta_{3} \right)} \\
 0 &  0 &  0 &  1 \\
\end{pmatrix}

Obtaining for $X$, $Y$ y $Z$:

\begin{equation} \label{eq:XeYeZe}
    \left.\begin{aligned}
        X_e & = \left(a_{2} \cos{\left(\theta_{2} \right)} + a_{3} \cos{\left(\theta_{2} - \theta_{3} \right)} + d_{1}\right) \cos{\left(\theta_{1} \right)} + T_X \\
        Y_e & = \left(a_{2} \cos{\left(\theta_{2} \right)} + a_{3} \cos{\left(\theta_{2} - \theta_{3} \right)} + d_{1}\right) \sin{\left(\theta_{1} \right)}       \\
        Z_e & = a_{1} + a_{2} \sin{\left(\theta_{2} \right)} + a_{3} \sin{\left(\theta_{2} - \theta_{3} \right)} - T_{Z}                                           \\
    \end{aligned}
    \right\}
\end{equation}